In [6]:
import json
from sklearn.externals import joblib
import os
import re

In [82]:
dataPath = '../data'
trainTfIdfPath = '../data/trainDataTfIdf'
tfIdfVectorizerPath = '../data/models/tfIdfVectorizer.pkl'
tfIdfNumberRagesPath = '../data/tfIdfNumberPages'
tfIdfRagesPath = '../data/tfidfPages'
tfIdfFeaturesPath = '../data/tfidfFeatutes'
docUrlsPath = '../data/urls.docs.txt'
textdataPath = '../data/textdata'
templateJson = '{:d}.json'

In [3]:
docDict = {}
with open(docUrlsName) as inputFile:
    for line in inputFile:
        number, url, path = line.strip().split('\t')
        docDict[int(number)] = (url, path)

pat = re.compile(r'\d+')
procNumbs = set(map(lambda x: int(pat.search(x).group(0)), os.listdir(textdataPath)))

# TFIDF

Создание обучающего множества для TfidfVectorizer

In [69]:
data = {"numbs":[], "strings":[]}
sizeMemory = 0
for numb in procNumbs:
    filename = templateJson.format(numb)
    path = '{:s}/{:s}'.format(textdataPath, filename)
    pageDict = json.load(open(path))
    text = pageDict['text'].get('p', [])
    text = '\n'.join(text)
    
    data["numbs"].append(numb)
    data["strings"].append(text)
    sizeMemory += len(text)
    if sizeMemory > 10 ** 9:
        break    

In [73]:
json.dump(data, open(trainTfIdfPath, 'w'))

In [74]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer

rusStopwords = stopwords.words('russian')
tokenizer = TweetTokenizer().tokenize
tfIdfVect = TfidfVectorizer(tokenizer=tokenizer, stop_words=rusStopwords, max_features=50000000)
tfIdfVect.fit(data["strings"])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=50000000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', '...гда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между'],
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f896ce3bac8>>,
        use_idf=True, vocabulary=None)

** Сохранение модели и результатов **

In [75]:
joblib.dump(tfIdfVect, tfIdfVectorizerPath)

['../data/models/tfIdfVectorizer.pkl']

In [76]:
result = tfIdfVect.transform(data["strings"])
json.dump(data["numbs"], open(tfIdfNumberRagesPath, 'w'))
json.dump(tfIdfVect.get_feature_names(), open(tfIdfFeaturesPath, 'w'))
joblib.dump(result, tfIdfRagesPath)

In [83]:
json.dump(tfIdfVect.get_feature_names(), open(tfIdfFeaturesPath, 'w'))

In [84]:
result.shape

(25216, 2147985)

In [85]:
help(result)

Help on csr_matrix in module scipy.sparse.csr object:

class csr_matrix(scipy.sparse.compressed._cs_matrix, scipy.sparse.sputils.IndexMixin)
 |  Compressed Sparse Row matrix
 |  
 |  This can be instantiated in several ways:
 |      csr_matrix(D)
 |          with a dense matrix or rank-2 ndarray D
 |  
 |      csr_matrix(S)
 |          with another sparse matrix S (equivalent to S.tocsr())
 |  
 |      csr_matrix((M, N), [dtype])
 |          to construct an empty matrix with shape (M, N)
 |          dtype is optional, defaulting to dtype='d'.
 |  
 |      csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
 |          where ``data``, ``row_ind`` and ``col_ind`` satisfy the
 |          relationship ``a[row_ind[k], col_ind[k]] = data[k]``.
 |  
 |      csr_matrix((data, indices, indptr), [shape=(M, N)])
 |          is the standard CSR representation where the column indices for
 |          row i are stored in ``indices[indptr[i]:indptr[i+1]]`` and their
 |          corresponding value